In [1]:
from bs4 import BeautifulSoup

In [2]:
#URL = '/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/a/A_cappella.htm'
URL= '/Users/ginevralarroux/Downloads/A_cappella.htm'
f=open(URL, 'r')
soup = BeautifulSoup(f, 'lxml')

In [3]:
#header
soup.h1
#
#title
soup.title 
soup.title.string #no html tags
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html dir="ltr" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="noindex,nofollow" name="robots"/>
<meta content="A cappella,1909,1980s,2007,2 Unlimited,ABC Family,A Cappella (album),Acappella (group),Accompaniment,All 4 One,Amish" name="keywords"/>
<link href="../../favicon.ico" rel="shortcut icon"/>
<link href="../../wp/w/Wikipedia_Text_of_the_GNU_Free_Documentation_License.htm" rel="copyright"/>
<title>A cappella</title>
<style media="screen,projection" type="text/css">/*<![CDATA[*/ @import "../../css/wp-monobook-main.css"; /*]]>*/</style>
<link href="../../css/wp-commonPrint.css" media="print" rel="stylesheet" type="text/css"/>
<!--[if lt IE 5.5000]><style type="text/css">@import "../../css/IE50Fixes.css";</style><![endif]-->
<!--[if IE 5.5000]><style type="te

In [4]:
for i, p in enumerate(soup.find_all("p")):
    as_in_p = p.find_all("a")
    for a in as_in_p:
        print(f"{i+1:<10} {a.text:<20} {a.get('href')}")

1          music                ../../wp/m/Music.htm
1          instrumental         ../../wp/m/Musical_instrument.htm
1          Renaissance          ../../wp/r/Renaissance_music.htm
2                               None
3          Gregorian chant      ../../wp/g/Gregorian_chant.htm
3          Renaissance          ../../wp/r/Renaissance_music.htm
4                               None
5          Amish                ../../wp/a/Amish.htm
5          Eastern Orthodox     ../../wp/e/Eastern_Orthodox_Church.htm
7                               None
8          Jewish               ../../wp/j/Judaism.htm
9                               None
11                              None
18         Beatles              ../../wp/t/The_Beatles.htm
21         Hindi                ../../wp/h/Hindi.htm
21         English              ../../wp/e/English_language.htm
22                              None
24         rap                  ../../wp/r/Rapping.htm


In [5]:
all_links = soup.find_all(['a', 'href'])
print('The webpage cointains {0} hypher links.'.format(len(all_links)))

The webpage cointains 23 hypher links.


In [6]:
paragraphs = soup.find_all('p')
print('The total number of paragraphs is {0}.'.format(len(paragraphs)))

The total number of paragraphs is 24.


In [7]:
links=[]
i=0

for p in paragraphs:
    i+=1
    all_links=p.find_all('a')
    for link in all_links:
        link_title=link.text
        link_paragraph=i
        links.append([link_title, link_paragraph])

import pandas as pd
link_positioning_df=pd.DataFrame(links, columns=['Link', 'Number of paragraph'])
link_positioning_df=link_positioning_df.loc[link_positioning_df['Link']!='']

In [8]:
link_positioning_df['Position']=round(link_positioning_df['Number of paragraph']/len(paragraphs), 2)
link_positioning_df

,Link,Number of paragraph,Position
0,music,1,0.04
1,instrumental,1,0.04
2,Renaissance,1,0.04
4,Gregorian chant,3,0.12
5,Renaissance,3,0.12
7,Amish,5,0.21
8,Eastern Orthodox,5,0.21
10,Jewish,8,0.33
13,Beatles,18,0.75
14,Hindi,21,0.88


In [9]:
URL = '/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/a/Aachen.htm'
f=open(URL, 'r')
ch = BeautifulSoup(f, 'lxml')

In [10]:
img = ch.find_all('img')
print('The total number of images is {0}.'.format(len(img)))

The total number of images is 24.


In [11]:
tables = ch.find_all('table')
print('The total number of tables is {0}.'.format(len(tables)))

The total number of tables is 2.


In [12]:
links=[]

for table in tables:
    if (table.img!=None):
        all_links= table.find_all('a')
        for link in all_links:
            link_title=link.text
            links.append(link_title)
        
link_in_table_df=pd.DataFrame(links)
link_in_table_df=link_in_table_df[link_in_table_df!='']
link_in_table_df.dropna()     

,0
2,Country
3,Germany


In [13]:
div = ch.find_all('div')
print(len(div))

links=[]

for d in div:
    if (d.find('a', class_="internal")):
        if (d.img):
            images=d.img.text
            print(images)
            caption=d.find('div', class_="thumbcaption")
            print(caption)
            if caption:
                all_links=caption.find_all('a')
                for link in all_links:
                    link_title=link.text
                    links.append(link_title)
            
link_in_div_df=pd.DataFrame(links)
link_in_div_df=link_in_div_df[link_in_table_df!='']
link_in_table_df.dropna()     

38

<div class="thumbcaption">
<div class="magnify" style="float:right"><a class="internal" href="../../images/39/3932.jpg.htm" title="Enlarge"><img alt="" height="11" src="../../images/0/1.png" width="15"/></a></div> Construction of Aix-la-Chapelle, by <!--del_lnk--> Jean Fouquet</div>

<div class="thumbcaption">
<div class="magnify" style="float:right"><a class="internal" href="../../images/39/3932.jpg.htm" title="Enlarge"><img alt="" height="11" src="../../images/0/1.png" width="15"/></a></div> Construction of Aix-la-Chapelle, by <!--del_lnk--> Jean Fouquet</div>

<div class="thumbcaption">
<div class="magnify" style="float:right"><a class="internal" href="../../images/39/3932.jpg.htm" title="Enlarge"><img alt="" height="11" src="../../images/0/1.png" width="15"/></a></div> Construction of Aix-la-Chapelle, by <!--del_lnk--> Jean Fouquet</div>

<div class="thumbcaption">
<div class="magnify" style="float:right"><a class="internal" href="../../images/39/3932.jpg.htm" title="Enlarge"><

,0
2,Country
3,Germany
